## Analisi dei dati

In [ ]:
import torch.nn as nn
import torch
import os
import shutil
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
from google.colab import drive

# Il dataset è presente su Google Drive come file zip
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Estrazione del contenuto del dataset
!unzip "/content/drive/MyDrive/Alzheimer_s_Dataset.zip" -d "/content/Alzheimer_s_Dataset"

In [ ]:
# Percorso del dataset decompresso
data = 'Alzheimer_s_Dataset'
data_dir_train = 'Alzheimer_s_Dataset/train'
data_dir_test = 'Alzheimer_s_Dataset/test'

In [ ]:
import os
import shutil

def merge_categories(parent_dir):
    # Unisci le categorie in 'Demented'
    for category in ['MildDemented', 'ModerateDemented', 'VeryMildDemented']:
        src_dir = os.path.join(parent_dir, category)
        dest_dir = os.path.join(parent_dir, 'Demented')
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)
        for file_name in os.listdir(src_dir):
            src_file_path = os.path.join(src_dir, file_name)
            new_file_name = f"{category}_{file_name}"
            dest_file_path = os.path.join(dest_dir, new_file_name)
            shutil.move(src_file_path, dest_file_path)

def remove_empty_folders(parent_dir):
    # Rimuovi le cartelle vuote
    for category in ['MildDemented', 'ModerateDemented', 'VeryMildDemented']:
        dir_path = os.path.join(parent_dir, category)
        if os.path.exists(dir_path) and len(os.listdir(dir_path)) == 0:
            os.rmdir(dir_path)

# Esegui le operazioni sulle cartelle di training e test
for directory in [data_dir_train, data_dir_test]:
    merge_categories(directory)
    remove_empty_folders(directory)

## Data Augmentation

In [ ]:
import random
import cv2

# Funzione di augmentation che esegue flip casuali e rotazioni
def augment_image(input_image_path, output_image_path):
    image = cv2.imread(input_image_path)

    # Flip casuale
    flip_hor = random.choice([True, False])
    if flip_hor:
        image = cv2.flip(image, 1)  # Flip orizzontale

    # Rotazione casuale
    angle = random.randint(-10, 10)
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    image = cv2.warpAffine(image, M, (cols, rows))

    # Salva l'immagine augmentata
    cv2.imwrite(output_image_path, image)

# Funzione per bilanciare le classi nel dataset di training
def balance_classes(data_type_dir, class_name, target_count=3000):
    class_dir = os.path.join(data_type_dir, class_name)
    current_count = len(os.listdir(class_dir))
    augmentations_needed = target_count - current_count

    if augmentations_needed > 0:
        for i in range(augmentations_needed):
            random_image_name = random.choice(os.listdir(class_dir))
            random_image_path = os.path.join(class_dir, random_image_name)
            augmented_image_name = f"aug_{i}_{random_image_name}"
            augmented_image_path = os.path.join(class_dir, augmented_image_name)
            augment_image(random_image_path, augmented_image_path)
    else:
        print(f"La classe {class_name} ha già {current_count} immagini, non sono necessarie ulteriori augmentations.")

In [ ]:
# Bilancia la classe 'Demented' nel dataset di training
balance_classes(data_dir_train, 'Demented')

# Bilancia la classe 'NonDemented' nel dataset di training (assumendo che esista già questa cartella)
balance_classes(data_dir_train, 'NonDemented')

## Pulizia della cache

In [ ]:
# Percorso della cartella .ipynb_checkpoints
checkpoints_path = os.path.join(data_dir_train, '.ipynb_checkpoints')

# Controlla se la cartella .ipynb_checkpoints esiste e, in tal caso, la elimina
if os.path.exists(checkpoints_path):
    shutil.rmtree(checkpoints_path)
    print(f"Cartella {checkpoints_path} rimossa.")

## Trasformazioni

In [ ]:
# Trasformazioni standard per il set di addestramento
standard_transforms = transforms.Compose([
    transforms.Resize((227, 227)),  # Ridimensionamento per AlexNet
    transforms.Grayscale(num_output_channels=1),  # Conversione in scala di grigi
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])  # Normalizzazione per un singolo canale
])